## Bullet Bonds

### Cash Flows

Consider a bond that was issued on January 1st 2020, and which matures on January 1st 2025. This bond pays 5% coupon which is paid once a year. 

In [22]:
from pyfian.fixed_income.bond import BulletBond

bond = BulletBond('2020-01-01', '2025-01-01', 5, 1, notional=100)

bond.payment_flow

{Timestamp('2021-01-01 00:00:00'): 5.0,
 Timestamp('2022-01-01 00:00:00'): 5.0,
 Timestamp('2023-01-01 00:00:00'): 5.0,
 Timestamp('2024-01-01 00:00:00'): 5.0,
 Timestamp('2025-01-01 00:00:00'): 105.0}

## Bond Day-Counting Conventions

These rules decide how to count days for calculating bond interest:

- **30/360**: Counts each month as 30 days, year as 360 days. Simple but less accurate. Used in some corporate bonds.
- **30E/360**: Like 30/360, but if a date is the 31st, it’s counted as the 30th. Used in Eurobonds.
- **Actual/Actual**: Counts real days in the period and year (365 or 366). Most accurate. Used in US Treasuries (ISDA) or international bonds (ICMA).
- **Actual/360**: Counts real days in the period, but uses a 360-day year. Common in money markets like loans.
- **Actual/365**: Counts real days in the period, uses a 365-day year. Used in some bond markets.
- **30/365**: Counts each month as 30 days, year as 365 days. Rare, used in specific financial contracts.
- **Actual/Actual-Bond**: Same as Actual/Actual (ICMA), counts real days in period and coupon period (e.g., half-year). Used in bond markets.

In [68]:
bond_1 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = '30/360')
bond_2 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = '30e/360')
#bond_3 = BulletBond('2020-01-01', '2025-01-01', 5, 1, notional=100, day_count_convention = 'actual/actual')
bond_4 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = 'actual/360')
bond_5 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = 'actual/365')
bond_6 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = '30/365')
bond_7 = BulletBond('2020-01-01', '2025-01-01', 5, 2, notional=100, day_count_convention = 'actual/actual-Bond')

print(bond_1.accrued_interest(settlement_date='2023-03-01'))
print(bond_2.accrued_interest(settlement_date='2023-03-01'))
print(bond_4.accrued_interest(settlement_date='2023-03-01'))
print(bond_5.accrued_interest(settlement_date='2023-03-01'))
print(bond_6.accrued_interest(settlement_date='2023-03-01'))
print(bond_7.accrued_interest(settlement_date='2023-03-01'))



0.41666666666666663
0.41666666666666663
0.4097222222222222
0.4041095890410959
0.410958904109589
1.6298342541436464


### Calculating Bond Price from Discount Curve

The price of a bond is determined by calculating the present value of its future cash flows, which include periodic coupon payments and the face value paid at maturity. A discount curve provides the discount factors used to compute these present values by accounting for the time value of money and interest rate risk. Each cash flow is multiplied by the discount factor corresponding to its payment time, and the sum of these discounted cash flows gives the bond's price.

For example, consider a bond with semi-annual coupon payments and a face value repaid at maturity. The discount curve, such as oneր
System: one defined by DummyCurve (provided in the conversation history), the bond price can be calculated as follows:

Cash Flows: For a 5-year bond with a 5% annual coupon rate (paid semi-annually) and a $100 face value, the cash flows are $2.50 every 6 months (at t=0.5, 1.0, ..., 4.5 years) and $102.50 at maturity (t=5.0).

Discounting: Each cash flow is multiplied by the discount factor from DummyCurve.discount_t(t) = 1 / (1 + 0.05 * t). For instance:

At t=0.5: $2.50 / (1 + 0.05 * 0.5) ≈ $2.439$.

At t=5.0: $102.50 / (1 + 0.05 * 5.0) = $82.0$.

Bond Price: The sum of these discounted cash flows (e.g., $2.439 + $2.381 + ... + $82.0 ≈ $102.066) is the bond's price.

This method ensures the bond price reflects the present value of all future payments, adjusted for the time value of money as defined by the discount curve.

In [76]:
class DummyCurve:
             def discount_t(self, t):
                return 1 / (1 + 0.05 * t)

print(bond_1.value_with_curve(DummyCurve())[1])

from IPython.display import Markdown

def display_cash_flows_table(cash_flow_dict):
    """
    Display a Markdown table of discounted cash flows from a dictionary of time:value pairs.
    
    Args:
        cash_flow_dict (dict): Dictionary with time points (float) as keys and
                              discounted cash flow values (float) as values.
    """
    # Start the Markdown table
    table = "## Discounted Cash Flows Table\n\n"
    table += "| Time (Years) | Discounted Cash Flow Value |\n"
    table += "|--------------|----------------------------|\n"
    
    # Sort times to ensure chronological order
    for time in sorted(cash_flow_dict.keys()):
        value = cash_flow_dict[time]
        # Round to 4 decimal places, unless the value is a whole number (e.g., 82.0)
        formatted_value = f"{value:.4f}" if value < 10 else f"{value:.4f}".rstrip('0').rstrip('.')
        table += f"| {time:.1f}          | {formatted_value:>26} |\n"
    
    # Display the table
    display(Markdown(table))

display_cash_flows_table(bond_1.value_with_curve(DummyCurve())[1])


{0.5: 2.439024390243903, 1.0: 2.380952380952381, 1.5: 2.3255813953488373, 2.0: 2.2727272727272725, 2.5: 2.2222222222222223, 3.0: 2.173913043478261, 3.5: 2.127659574468085, 4.0: 2.0833333333333335, 4.5: 2.040816326530612, 5.0: 82.0}


## Discounted Cash Flows Table

| Time (Years) | Discounted Cash Flow Value |
|--------------|----------------------------|
| 0.5          |                     2.4390 |
| 1.0          |                     2.3810 |
| 1.5          |                     2.3256 |
| 2.0          |                     2.2727 |
| 2.5          |                     2.2222 |
| 3.0          |                     2.1739 |
| 3.5          |                     2.1277 |
| 4.0          |                     2.0833 |
| 4.5          |                     2.0408 |
| 5.0          |                         82 |
